In [ ]:
from imports_scgen_local import * #imports_scgen_local have to be in same folder

In [ ]:
pre = 'MH89'
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
_,base_table_path_MH78,_,_ = h.return_local_paths(drive = drive,pre = "MH78",add_path = False)

In [ ]:
df_magnitude_latent = pd.read_csv(os.path.join(base_table_path_MH78,'MH78' + '_df_magnitude_latent.csv'),index_col=0)

In [ ]:
df_shift_cos = pd.read_csv(os.path.join(base_table_path_MH78,'MH78' + 'df_cos.csv'),index_col=0)

In [ ]:
df_magnitude_human = df_magnitude_latent[df_magnitude_latent['species'] == 'human']
df_magnitude_cyno = df_magnitude_latent[df_magnitude_latent['species'] == 'cyno']

celltypes = ['B',
 'CD14 Mono',
 #'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']

all_species = ['cyno','human']
all_conditions = ['00hr','06hr','24hr']
n_latent = 10
shifts_within_species = ['00hr->06hr','00hr->24hr','06hr->24hr']


In [ ]:
model_path = os.path.join(*["F:\\","monkey_IZI","analysis","cluster_runs","workspace-runVAE",
                            "output","MH49_c","models"])

In [ ]:
warnings.filterwarnings('ignore')
df_shifts = {}
for model_dir_name in celltypes:
    load_string = os.path.join(model_path,model_dir_name,'MH49_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    model.adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    model.adata.obs = h.make_annotation_azimut1_5_scanvi_v2(model.adata)
    latent_adata = hscg.get_latent_representation_object(model=model,adata=model.adata)

    # prepare anndata objects for shifting
    adatas_latent_dict = {}
    for species in all_species:
        for condition in all_conditions:
            adatas_latent_dict[species + '_' +condition] = h.filter_multiple_adata_obs(adata=latent_adata,
                                                            col_names = ['species','timepoint'],
                                                            val_names=[species,condition])

    delta_df_human = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='human',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    delta_df_cyno = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='cyno',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    shifts_within_species = ['00hr->06hr','00hr->24hr','06hr->24hr']
    delta_df_human.to_csv(os.path.join(base_table_path,pre + '_' + model_dir_name + '_delta_df_human.csv'))
    delta_df_cyno.to_csv(os.path.join(base_table_path,pre + '_' + model_dir_name + '_delta_df_cyno.csv'))
    df_shift = pd.DataFrame(columns = ['shift_human','shift_cyno','euclidean_distance','cosine_similarity'])
    for human_shift in shifts_within_species:
        for cyno_shift in shifts_within_species:
            delta_cyno = delta_df_cyno.loc[human_shift].filter(regex='^latent_dim_').values
            delta_human = delta_df_human.loc[cyno_shift].filter(regex='^latent_dim_').values
            eucl_dist_diff = np.linalg.norm(delta_human - delta_cyno)
            cos_sim = cosine_similarity(delta_cyno[np.newaxis,:],delta_human[np.newaxis,:])
            df_shift = df_shift.append(pd.DataFrame([human_shift,cyno_shift,eucl_dist_diff,cos_sim[0][0]],
                                                    index = ['shift_human','shift_cyno','euclidean_distance','cosine_similarity']).T)
            df_shift.to_csv(os.path.join(base_table_path, pre + '_' +'shift_eucl_dist_' + model_dir_name + '.csv'))
    df_shifts[model_dir_name] = df_shift

In [ ]:
max_lim_plot = max([np.max(df_magnitude_human['magnitude']),np.max(df_magnitude_cyno['magnitude'])]) + 1
# Colors for arrows
col_human_arrow =  "#e35e28"
col_cyno_arrow ="#31c7ba"

# Create subplots
fig, axes = plt.subplots(len(celltypes), len(shifts_within_species), figsize=(14, 21))  # Adjust the figure size as needed
fig.subplots_adjust(hspace=10, wspace=10) 
for i, celltype in enumerate(celltypes):
    for j, shift_within_species in enumerate(shifts_within_species):
        ax = axes[i, j]
        if shift_within_species == '00hr->06hr':
            cos_sim = df_shift_cos[celltype].loc[shift_within_species]  # Example cosine similarity for this shift

            mag_human = df_magnitude_human[(df_magnitude_human['shift'] == shift_within_species)
                                           &(df_magnitude_human['celltype'] == celltype)]['magnitude'].values[0]

            mag_cyno = df_magnitude_cyno[(df_magnitude_cyno['shift'] == shift_within_species)
                                         &(df_magnitude_cyno['celltype'] == celltype)]['magnitude'].values[0]

            final_base_vector,final_rotated_vector = h.get_2D_vectors(norm_base_vector=mag_human,
                                                                    norm_rotated_vector=mag_cyno,
                                                                    cosine_sim=cos_sim,
                                                                    base_vector = np.array([1,0]))


            _,final_most_unsimilar_vector = h.get_2D_vectors(norm_base_vector=mag_human,
                                                                    norm_rotated_vector=mag_human,
                                                                    cosine_sim=-1,
                                                                    base_vector = final_base_vector)
        else:
            delta_df_human_ct = pd.read_csv(os.path.join(base_table_path,pre + '_' + celltype + '_delta_df_human.csv'),index_col=0)
            extra_shift = cosine_similarity(
                delta_df_human_ct.loc['00hr->06hr'].filter(regex='^latent_dim_').values[np.newaxis,:],
                delta_df_human_ct.loc[shift_within_species].filter(regex='^latent_dim_').values[np.newaxis,:])[0][0]
            cos_sim = df_shift_cos[celltype].loc[shift_within_species]  # Example cosine similarity for this shift

            mag_human = df_magnitude_human[(df_magnitude_human['shift'] == shift_within_species)
                                           &(df_magnitude_human['celltype'] == celltype)]['magnitude'].values[0]

            mag_cyno = df_magnitude_cyno[(df_magnitude_cyno['shift'] == shift_within_species)
                                         &(df_magnitude_cyno['celltype'] == celltype)]['magnitude'].values[0]

            _,start_base_vector = h.get_2D_vectors(norm_base_vector=1,
                                                                    norm_rotated_vector=1,
                                                                    cosine_sim=extra_shift,
                                                                    base_vector = np.array([1,0]))

            final_base_vector,final_rotated_vector = h.get_2D_vectors(norm_base_vector=mag_human,
                                                                    norm_rotated_vector=mag_cyno,
                                                                    cosine_sim=cos_sim,
                                                                    base_vector = start_base_vector)


            _,final_most_unsimilar_vector = h.get_2D_vectors(norm_base_vector=mag_human,
                                                                    norm_rotated_vector=mag_human,
                                                                    cosine_sim=-1,
                                                                    base_vector = final_base_vector)
            
            
        # Get data for this subplot
        #cos_sim = df_shift_cos[celltype].loc[shift_within_species]  # Example cosine similarity for this shift
        #mag_human = df_magnitude_human[(df_magnitude_human['shift'] == shift_within_species)&(df_magnitude_human['celltype'] == celltype)]['magnitude'].values[0]
        #mag_cyno = df_magnitude_cyno[(df_magnitude_cyno['shift'] == shift_within_species)&(df_magnitude_cyno['celltype'] == celltype)]['magnitude'].values[0]

        # Compute angle and vectors
        #angle = np.arccos(cos_sim)
        #vec1 = np.array([mag_human, 0])
        #vec2 = np.array([mag_cyno * np.cos(angle), mag_cyno * np.sin(angle)])

        # Plot vectors
        ax.quiver(0, 0, final_base_vector[0], final_base_vector[1], angles='xy', scale_units='xy',
                  scale=1, color=col_human_arrow, label='Human', width=0.025,headwidth=2.5, headlength=3.5)
        ax.quiver(0, 0, final_most_unsimilar_vector[0], final_most_unsimilar_vector[1], angles='xy', scale_units='xy',
                  scale=1, color="#e6e6e6", label='-Human', width=0.025,headwidth=2.5, headlength=3.5)
        ax.quiver(0, 0, final_rotated_vector[0], final_rotated_vector[1], angles='xy', scale_units='xy',
                  scale=1, color=col_cyno_arrow, label='Cyno', width=0.025,headwidth=2.5, headlength=3.5)
       
        
        

        # Set limits
        ax.set_xlim(-max_lim_plot-3, max_lim_plot+3)
        ax.set_ylim(-max_lim_plot-3, max_lim_plot+3)

        # Title and remove axis labels/ticks
        ax.set_title(str(celltype) + ' : ' + str(shift_within_species))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.legend(loc='lower right')

# Adjust layout
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_rel_arrow_plot.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()